# Ray Casting: 3D gaming with 2D logic



[original motivation and why this is cool]

We're going to keep the imports as lightweight as possible to stick with the spirit of the original implementation. 

Python vs C, machine limitations

In [1]:
# Grab our basic operations and drawing functions
import math
import sys

import pyglet
from pyglet.gl import *
from pyglet.window import key

[Basics of the algorithm]

In [2]:
# Visualization tool for the ray casting calculations

[what we need]

1) Set up game controls and player state
2) Write the tools we need to cast a ray from the player
3) Render the environment as we move using our ray casting functions

## Step 1: Create our game environment

We first need a map to move around in. Because this game run on 2D logic, our map can simply be a 2D array of **wall** or **no wall**.

In [3]:
from typing import List

# This is what our 2D map representation will look like
game_map: List[str] = [
        '################',
        '#     # #    # #',
        '# ###     ## # #',
        '#   ### #  #   #',
        '# #   # #### ###',
        '# ### #  ##    #',
        '#   #       ## #',
        '################'
    ]

Now let's start constructing our game environment - this class will handle x, y, z. We will also import a nifty decorator that lets us build the class iteratively over the rest of this notebook.

In [4]:
# Grab a helper to update our game class throughout the notebook
from utils.class_updates import update_class
context = globals()

# Define the initial version of the game environment
class GameEnvironment:

    # Set up the player's view within the map
    fov: float = math.pi / 2.7 # Player field of view
    half_fov = fov * 0.5
    angle_step: float = fov / 160 # The angle between rays
    wall_height: float = 100.0 # Wall height in pixels at one distance unit

    def point_is_wall(self, x: float, y: float) -> bool:
        """Determine if a given point in the map is in a wall"""
        return True if self.game_map[math.floor(x)][math.floor(y)] == '#' else False

Now we need a player that can move around the game environment.

In [5]:
from dataclasses import dataclass

@dataclass
class PlayerState:
    x: float
    y: float
    angle: float
    step_size: float

@update_class(context)
class GameEnvironment:

    def __init__(self, player: PlayerState, game_map: List[str]):
        self.player = player
        self.game_map = game_map

    def update_player(self, up: bool=False, down: bool=False, 
                            left: bool=False, right: bool=False):
        """Update the player state based on the keyboard input"""
        prev_position = (self.player.x, self.player.y)

        # Calculate our step along player angle
        x_step = math.cos(self.player.angle) * self.player.step_size
        y_step = -math.sin(self.player.angle) * self.player.step_size

        if up:
            self.player.x += x_step
            self.player.y += y_step
        if down:
            self.player.x -= x_step
            self.player.y -= y_step
        if right:
            self.player.angle -= self.player.step_size
        if left:
            self.player.angle += self.player.step_size

        # Only step if we aren't going to end up inside of a wall
        if self.point_is_wall(self.player.x, self.player.y):
            (self.player.x, self.player.y) = prev_position

## Step 2: Write the ray casting tools

[horizontal intersection and vertical intersection]

In [6]:
@update_class(context)
class GameEnvironment:

    def player_delta(self, a: float, b: float):
        return math.sqrt((a*a) + (b*b))

    def cast_horizontal_ray(self, view_angle: float):
        # Determine if the player angle is "facing up" within the map
        facing_up: bool = abs(math.floor(view_angle / math.pi) % 2.0) != 0.0

        # Find the ray extension values
        dy = 1.0 if facing_up else -1.0
        dx = -dy / math.tan(view_angle)

        first_step = True
        while True:
            # Move the ray forward one step relative to the player
            if first_step:
                # Determine the first grid lines the ray intersects with
                # This is where our ray starts
                if facing_up:
                    ray_y = math.ceil(self.player.y) - self.player.y
                else:
                    ray_y = math.floor(self.player.y) - self.player.y
                ray_x = -ray_y / math.tan(view_angle)
                first_step = False
            else:
                ray_x += dx
                ray_y += dy

            # Determine the ray's current absolute map coordinates
            ray_x_world = ray_x + self.player.x
            ray_y_world = ray_y + self.player.y
            if not facing_up:
                ray_y_world -= 1.0
            
            # Check if our ray has hit a wall
            print("Horizontal x, y", ray_x_world, ray_y_world)
            print("Map location", math.floor(ray_x_world), math.floor(ray_y_world))
            if self.point_is_wall(ray_x_world, ray_y_world): 
                break

        return self.player_delta(ray_x, ray_y)
        

    def cast_vertical_ray(self, view_angle: float):
        # Determine if the player angle is "facing right" within the map
        facing_right: bool = abs(math.floor((view_angle - math.pi/2.0) / math.pi) % 2.0) != 0.0

        # Find the ray extension values
        dx = 1.0 if facing_right else -1.0
        dy = dx * -math.tan(view_angle)

        first_step = True
        while True:
            # Move the ray forward one step relative to the player
            if first_step:
                # Determine the first grid lines the ray intersects with
                # This is where our ray starts
                if facing_right:
                    ray_x = math.ceil(self.player.x) - self.player.x
                else:
                    ray_x = math.floor(self.player.x) - self.player.x
                ray_y = -ray_x * math.tan(view_angle)
                first_step = False
            else:
                ray_x += dx
                ray_y += dy

            # Determine the ray's current absolute map coordinates
            ray_y_world = ray_y + self.player.y
            ray_x_world = ray_x + self.player.x
            if not facing_right:
                ray_x_world -= 1.0
            
            # Check if our ray has hit a wall
            if self.point_is_wall(ray_x_world, ray_y_world): 
                break

        return self.player_delta(ray_x, ray_y)

## Step 3: Run and render the game

In [7]:
@update_class(context)
class GameEnvironment:
    def get_view(self) -> List[float]:
        """TODO: summary

        Returns:
            List[float]: A list of the wall heights to render across the player's view
        """
        # Determine where the player is "looking"
        start_angle: float = self.player.angle + self.half_fov

        walls: List[float] = [0] * 160
        for wall_index in range(len(walls)):
            # Determine the wall's angle w.r.t. the player's view
            view_angle: float = start_angle - float(wall_index) * self.angle_step

            # Get the distance to the closest wall intersection
            horizontal_delta = self.cast_horizontal_ray(view_angle)
            vertical_delta = self.cast_vertical_ray(view_angle)
            closest_wall_delta: float = min(horizontal_delta, vertical_delta)
            print(wall_index, closest_wall_delta)

            # Convert the closest ray intersection distance to wall height
            walls[wall_index] = self.wall_height / closest_wall_delta
            print("Wall: ", wall_index, walls[wall_index])
        
        return walls
    
    def render_view(self, walls: List[float]):
        batch = pyglet.graphics.Batch()

        batch.draw()

    def run_game(self):
        # Start the game window with the initial player view
        window = pyglet.window.Window(vsync=False, width=160, height=160, title="Ray Casting")
        walls: List[float] = []
        # self.get_view()

        @window.event
        def on_draw():
            window.clear()
            self.render_view(walls)

        @window.event
        def on_key_press(symbol, modifier):
            if symbol == pyglet.window.key.ESCAPE:
                window.close()
            
            if symbol == pyglet.window.key.UP:
                self.update_player(up=True)
            elif symbol == pyglet.window.key.DOWN:
                self.update_player(down=True)
            elif symbol == pyglet.window.key.LEFT:
                self.update_player(left=True)
            elif symbol == pyglet.window.key.RIGHT:
                self.update_player(right=True)

            walls = self.get_view()

        pyglet.app.run()


You can see everything we've added to this class assembled in [one file here](./utils/GameEnvironment.py).

In [8]:
# Set up our initial player state and game map
player: PlayerState = PlayerState(x = 2.5,
                                  y = 2.5,
                                  angle = 0.0,
                                  step_size = 0.045)
game_map: List[str] = ['################',
                  '#            # #',
                  '#         ## # #',
                  '#       #  #   #',
                  '# #   # #### ###',
                  '# ### #  ##    #',
                  '#   #       ## #',
                  '################']

# Run the game
game = GameEnvironment(player, game_map)
game.get_view()

Horizontal x, y 3.2602130672606418 1.0
Map location 3 1
Horizontal x, y 4.780639201781926 0.0
Map location 4 0
0 2.729709722425537
Wall:  0 36.63393187138706
Horizontal x, y 3.2723895809287784 1.0
Map location 3 1
Horizontal x, y 4.817168742786335 0.0
Map location 4 0
1 2.760302697630462
Wall:  1 36.227910832331325
Horizontal x, y 3.2848427885587452 1.0
Map location 3 1
Horizontal x, y 4.854528365676235 0.0
Map location 4 0
2 2.7917384950553665
Wall:  2 35.81997388978826
Horizontal x, y 3.2975835928099353 1.0
Map location 3 1
Horizontal x, y 4.892750778429805 0.0
Map location 4 0
3 2.8240496255690055
Wall:  3 35.4101426173952
Horizontal x, y 3.3106234680931816 1.0
Map location 3 1
Horizontal x, y 4.931870404279545 0.0
Map location 4 0
4 2.8572703167902676
Wall:  4 34.99843868897067
Horizontal x, y 3.3239744987090445 1.0
Map location 3 1
Horizontal x, y 4.9719234961271335 0.0
Map location 4 0
5 2.8914366274752403
Wall:  5 34.584883877368085
Horizontal x, y 3.3376494200779447 1.0
Map loc

IndexError: list index out of range

**Credits:**

This notebook was inspired by [Grant Handy's blog post here](https://grantshandy.github.io/posts/raycasting/).

He implements the ray casting algorithm and demo game in Rust with the goal of optimizing the total memory used (the same goal that the original developers had as we discussed above).  Jump over to see the classic implementation with more discussion of other optimization tricks.